In [1]:
import requests
import re
import json
import csv
import time
import datetime

## Purchase History

In [2]:
def purchase_data(item_ID, transactions, servers):
    URL = 'https://universalis.app/api/history/'
    cross_server_data = []
    
    for server in servers:
        req = requests.get(URL+str(server)+'/'+str(item_ID)+'?entries='+str(transactions))
        sales = json.loads(req.text)
        output = []
        
        # add exception-handling for servers with 0 transactions
        try:
            for i in sales['entries']:
                row = [sales['itemID'], sales['worldName'], sales['regularSaleVelocity'],sales['nqSaleVelocity'], 
                       sales['hqSaleVelocity'], i['pricePerUnit'], i['quantity'], i['hq'],
                       datetime.datetime.utcfromtimestamp(i['timestamp']).strftime("%m/%d/%Y,%H:%M:%S")]

                # add exception-handling for entries where buyerName didn't exist
                try:
                    row.insert(len(row)-1, i['buyerName'])
                except:
                    row.insert(len(row)-1, "na")

                output.append(row)

        except:
            pass
        
        cross_server_data.append(output)
        
    return cross_server_data

## Current Listings

In [3]:
def current_listings(item_ID, servers, listings, entries):
    URL = 'https://universalis.app/api/'
    cross_server_data = []
    
    for server in servers:
        req = requests.get(URL+str(server)+'/'+str(item_ID)+'?listings='+str(listings)+'&entries='+f'{entries}')
        current = json.loads(req.text)
        
        output = []
        # add exception-handling for servers with 0 listings
        try:
            for i in current['listings']:
                row = [current['itemID'], current['worldName'], 
                       current['regularSaleVelocity'],current['nqSaleVelocity'], current['hqSaleVelocity'], 
                       current['currentAveragePrice'], current['currentAveragePriceNQ'], current['currentAveragePriceHQ'],
                       current['minPrice'], current['minPriceNQ'], current['minPriceHQ'],
                       current['maxPrice'], current['maxPriceNQ'], current['maxPriceHQ'], 
                       i['pricePerUnit'], i['quantity'], i['hq'], i['isCrafted'], i['retainerName'],
                       datetime.datetime.utcfromtimestamp(i['lastReviewTime']).strftime("%m/%d/%Y,%H:%M:%S")]
                
                try:
                    row.insert(datetime.datetime.utcfromtimestamp(current['lastUploadTime']).strftime("%m/%d/%Y,%H:%M:%S"))
                except:
                    row.insert(len(row)+1, 'na')
            
                output.append(row)
            
        except:
            pass
               
        cross_server_data.append(output)
        
    return cross_server_data

## Execute

### Write Purchase History to csv

In [4]:
#all_servers =  ['Balmung','Brynhildr','Coeurl','Diabolos','Goblin','Malboro','Mateus','Zalera']
# 26108
# 23893 bathroom floor tiles (last entry feb2022)
# 36074 thavnairian chai (last entry 6/11/2022, as of 6/19/2022, missing historical data)
# 36167 lunar adamantite ore (last entry jun2022)
# 37282 Calamari Ripieni (last entry jun2022)

item_ID = 36074
transactions = None
servers =  ['Mateus']
filename = str(item_ID)+'_Purchase_History.csv'

with open(filename,'w',encoding='utf-8',newline="") as f:
    fr = csv.writer(f)
    #Write Header
    fr.writerow(('itemID', 'Server', 'regularSaleVelocity','nqSalesVelocity',
                 'HQSalesVelocity', 'Price', 'Quantity','HQ', 'buyerName','Timestamp'))

    #Write rows
    for i in range(len(purchase_data(item_ID, transactions, servers))):
        for row in purchase_data(item_ID, transactions, servers)[i]:
            fr.writerow(row)

### Write Current Listings to csv

In [5]:
item_ID = 36074
listings = None
entries = None
servers = ['Balmung','Brynhildr','Coeurl','Diabolos','Goblin','Malboro','Mateus','Zalera']
filename = str(item_ID)+'_listings.csv'

with open(filename,'w',encoding='utf-8',newline="") as f:
    fr = csv.writer(f)
    #Write Header
    fr.writerow(('itemID', 'Server', 
                 'regularSaleVelocity','nqSalesVelocity','HQSalesVelocity', 
                 'currentAvePrice', 'currentAvePriceNQ', 'currentAvePriceHQ',
                 'minPrice', 'minPriceNQ', 'minPriceHQ',
                 'maxPrice', 'maxPriceNQ', 'maxPriceHQ',
                 'Price', 'Quantity','HQ', 'isCrafted', 'retainerName','lastReviewTime', 'lastUploadTime'))

    #Write rows
    for i in range(len(current_listings(item_ID, servers, listings, entries))):
        for row in current_listings(item_ID, servers, listings, entries)[i]:
            fr.writerow(row)